In [1]:
import sys
print(sys.executable)

/mnt/d/AWS/fraud-detection/.env/bin/python3


### Prepare data

In [2]:
import boto3
import os

from sagemaker.local import LocalSession

sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}


os.makedirs("local_output", exist_ok=True)


/mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/04/25 16:06:35] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=672112;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=60702;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/botocore/credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alber/.config/sagemaker/config.yaml


[04/04/25 16:06:52] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=181038;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=695812;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/botocore/credentials.py#1352\1352]8;;\

In [5]:
from sagemaker import image_uris

region = boto3.Session().region_name
version = "1.3-1"

image_uri = image_uris.retrieve("xgboost", region, version)
image_uri

[04/04/25 16:07:58] INFO     Ignoring unnecessary instance type: None.                            ]8;id=676161;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=463306;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1'

In [6]:
# demo create data

import pandas as pd
import numpy as np

# Crear datos de ejemplo
data = np.random.rand(100, 10)  # 100 filas, 10 características
labels = np.random.randint(2, size=100)  # 100 valores binarios

df = pd.DataFrame(data, columns=[f"feature_{i}" for i in range(10)])
df["label"] = labels

# Guardar los datos como CSV
os.makedirs("data", exist_ok=True)
df.to_csv("data/train.csv", index=False, header=False)
df.to_csv("data/validation.csv", index=False, header=False)

### Initializate model

In [8]:
from sagemaker.estimator import Estimator

role = "arn:aws:iam::390844780979:role/SageMakerRole"

hyperparameters = {
    'max_depth': 5,
    'eta': 0.2,
    'objective': 'reg:squarederror',
    'num_round': 50
}

estimator = Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type="local",
    output_path="file://local_output",
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters
    
)



[04/04/25 16:08:23] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=383168;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=733091;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/botocore/credentials.py#1352\1352]8;;\

In [11]:
from urllib.parse import urlparse
import os

train_uri = 'file:///mnt/d/AWS/fraud-detection/src/train/data/train.csv'
valid_uri = 'file:///mnt/d/AWS/fraud-detection/src/train/data/validation.csv'

print(os.path.exists(urlparse(train_uri).path))
print(os.path.exists(urlparse(valid_uri).path))

True
True


### Training model

In [3]:
from sagemaker.mxnet import MXNet

# Configure an MXNet Estimator (no training happens yet)
mxnet_estimator = MXNet('train.py',
                        role='SageMakerRole',
                        instance_type='local',
                        instance_count=1,
                        framework_version='1.2.1')

# In Local Mode, fit will pull the MXNet container Docker image and run it locally
mxnet_estimator.fit("s3://fraud-detection-data-ars/train_data/train.csv")

# Alternatively, you can train using data in your local file system. This is only supported in Local mode.
#mxnet_estimator.fit('file:///tmp/my_training_data')

# Deploys the model that was generated by fit() to local endpoint in a container
#mxnet_predictor = mxnet_estimator.deploy(initial_instance_count=1, instance_type='local')

# Serializes data and makes a prediction request to the local endpoint
#response = mxnet_predictor.predict(data)

# Tears down the endpoint container and deletes the corresponding endpoint configuration
#mxnet_predictor.delete_endpoint()

# Deletes the model
#mxnet_predictor.delete_model()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 from sagemaker.mxnet import MXNet                                                           │
│    2                                                                                             │
│    3 # Configure an MXNet Estimator (no training happens yet)                                    │
│ ❱  4 mxnet_estimator = MXNet('train.py',                                                         │
│    5 │   │   │   │   │   │   role='SageMakerRole',                                               │
│    6 │   │   │   │   │   │   instance_type='local',                                              │
│    7 │   │   │   │   │   │   instance_count=1,                                                   │
│                                                                                                  │
│ /mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/mxnet/estimator.py:159 in │
│ __init__                                                                                         │
│                                                                                                  │
│   156 │   │   instance_type = renamed_kwargs(                                                    │
│   157 │   │   │   "train_instance_type", "instance_type", kwargs.get("instance_type"), kwargs    │
│   158 │   │   )                                                                                  │
│ ❱ 159 │   │   validate_version_or_image_args(framework_version, py_version, image_uri)           │
│   160 │   │   if py_version == "py2":                                                            │
│   161 │   │   │   logger.warning(                                                                │
│   162 │   │   │   │   python_deprecation_warning(self._framework_name, defaults.LATEST_PY2_VER   │
│                                                                                                  │
│ /mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/fw_utils.py:1174 in       │
│ validate_version_or_image_args                                                                   │
│                                                                                                  │
│   1171 │   │   │   None.                                                                         │
│   1172 │   """                                                                                   │
│   1173 │   if (framework_version is None or py_version is None) and image_uri is None:           │
│ ❱ 1174 │   │   raise ValueError(                                                                 │
│   1175 │   │   │   "framework_version or py_version was None, yet image_uri was also None. "     │
│   1176 │   │   │   "Either specify both framework_version and py_version, or specify image_uri.  │
│   1177 │   │   )                                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: framework_version or py_version was None, yet image_uri was also None. Either specify both 
framework_version and py_version, or specify image_uri.

In [9]:
from sagemaker.inputs import TrainingInput

estimator.fit(
    {
        "train": TrainingInput(
            "s3://fraud-detection-data-ars/train_data/train.csv",
            content_type="csv"
        ),
        "validation": TrainingInput(
            "s3://fraud-detection-data-ars/train_data/validation.csv",
            content_type="csv"
            
        )
    }
)

[04/04/25 16:08:34] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=719404;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=576100;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=67495;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=64452;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-04-04-15-08-34-232                                             

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=247822;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=931504;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     'Docker Compose' found using Docker CLI.                                  ]8;id=785963;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=53783;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/image.py#168\168]8;;\

                    INFO     Starting training job                                             ]8;id=929550;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/local_session.py\local_session.py]8;;\:]8;id=671673;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/local_session.py#215\215]8;;\

...

                    INFO     docker compose file:                                                      ]8;id=653950;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=729378;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/image.py#781\781]8;;\
                             networks:                                                                             
                               sagemaker-local:                                                                    
                                 name: sagemaker-local                                                             
                             services:                                                                             
                               algo-1-9kaf0:                                                                       
                                 command: train                                                                    
                                 container_name: dhf9etdhc3-algo-1-9kaf0                                           
                                 environment:                                                                      
                                 - '[Masked]'                                                                      
                                 - '[Masked]'                                                                      
                                 - '[Masked]'                                                                      
                                 - '[Masked]'                                                                      
                                 - '[Masked]'                                                                      
                                 image:                                                                            
                             683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1                  
                                 networks:                                                                         
                                   sagemaker-local:                                                                
                                     aliases:                                                                      
                                     - algo-1-9kaf0                                                                
                                 stdin_open: true                                                                  
                                 tty: true                                                                         
                                 volumes:                                                                          
                                 - /tmp/tmp8nawbdj8/algo-1-9kaf0/output:/opt/ml/output                             
                                 - /tmp/tmp8nawbdj8/algo-1-9kaf0/input:/opt/ml/input                               
                                 - /tmp/tmp8nawbdj8/algo-1-9kaf0/output/data:/opt/ml/output/data                   
                                 - /tmp/tmp8nawbdj8/model:/opt/ml/model                                            
                                 - /tmp/tmpqwasi1df:/opt/ml/input/data/train                                       
                                 - /tmp/tmpqdzzc182:/opt/ml/input/data/validation                                  
                             version: '2.3'                                                                        
                                                                                                                   

                    INFO     docker command: docker compose -f /tmp/tmp8nawbdj8/docker-compose.yaml up ]8;id=231711;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=533626;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/local/image.py#805\805]8;;\
                             --build --abort-on-container-exit                                                     

time="2025-04-04T16:08:36+01:00" level=warning msg="/tmp/tmp8nawbdj8/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
time="2025-04-04T16:08:36+01:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmp8nawbdj8\".\nSet `external: true` to use an existing network"
 Container dhf9etdhc3-algo-1-9kaf0  Creating
 Container dhf9etdhc3-algo-1-9kaf0  Created
Attaching to dhf9etdhc3-algo-1-9kaf0
dhf9etdhc3-algo-1-9kaf0  | [2025-04-04 15:08:39.039 b8294b5138bd:1 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
dhf9etdhc3-algo-1-9kaf0  | [2025-04-04 15:08:39.086 b8294b5138bd:1 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
dhf9etdhc3-algo-1-9kaf0  | [2025-04-04:15:08:39:INFO] Imported framework sagemaker_xgboost_container.training
dhf9etdhc3-algo-1-9kaf0  | [2025-04-04:15:08:39:INFO] Failed to

[04/04/25 16:08:40] ERROR    Please check the troubleshooting guide for common errors:              ]8;id=601211;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=48774;file:///mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/session.py#1050\1050]8;;\
                             https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-t                
                             roubleshooting.html#sagemaker-python-sdk-troubleshooting-create-traini                
                             ng-job                                                                                

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 from sagemaker.inputs import TrainingInput                                                  │
│    2                                                                                             │
│ ❱  3 estimator.fit(                                                                              │
│    4 │   {                                                                                       │
│    5 │   │   "train": TrainingInput(                                                             │
│    6 │   │   │   "s3://fraud-detection-data-ars/train_data/train.csv",                           │
│                                                                                                  │
│ /mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_loggi │
│ ng.py:167 in wrapper                                                                             │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_loggi │
│ ng.py:138 in wrapper                                                                             │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /mnt/d/AWS/fraud-detection/.env/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context │
│ .py:346 in wrapper                                                                               │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                    